# NeuralLogicRec evaluation

In [2]:
import utils.common
import evaluation
import importlib
import numpy as np
import tensorflow as tf
import models.MatrixFactorization
import models.BPR
import models.ConstraintAutoRec
import models.NeuralLogicRec
import pandas as pd
from models.NeuralLogicRec import item_cf, user_cf, diversity_constraint, Constraint, And, Or, Implies, Forall, Not, Equiv
importlib.reload(utils.common)
importlib.reload(evaluation)
importlib.reload(models.NeuralLogicRec)

<module 'models.NeuralLogicRec' from '/home/ec2-user/SageMaker/rs-ml-with-constraints/models/NeuralLogicRec.py'>

In [3]:
eval_ml = evaluation.Evaluation(utils.common.movie_lens)
ml_evals = list()
ev_test = evaluation.Evaluation(utils.common.ml_small)

## Definition of constraints

In [6]:
constraints = list()
constraints.append(Constraint(weight=0.2, formula=item_cf))
constraints.append(Constraint(weight=0.2, formula=user_cf))
@tf.function
def likes_equiv(model, outputs):
    return Forall(Equiv(outputs['rec'], outputs['likes']))
constraints.append(Constraint(weight=0.8, formula=likes_equiv))

popularity = np.load(utils.common.movie_lens['train']['item_frequency'])
pop = tf.convert_to_tensor(popularity.squeeze(), tf.float32)
@tf.function
def novelty_constraint(model, outputs):
    return Forall(Implies(pop, Not(outputs['rec'])))
constraints.append(Constraint(weight=0.3, formula=novelty_constraint))
constraints.append(Constraint(weight=0.3, formula=diversity_constraint))

## Models for MovieLens data

In [77]:
nlr_ml = models.NeuralLogicRec.NLR(utils.common.movie_lens['user'], utils.common.movie_lens['dimensions'], epochs=1, constraints=constraints, mode='ae')
nlr_history = list()

In [81]:
for e in range(4):
    nlr_ml.train(utils.common.load_dataset(utils.common.movie_lens), utils.common.movie_lens['train']['records'])
    ev = eval_ml.evaluate_single_thread(nlr_ml)
    print(ev)
    nlr_history.append(ev)

Epoch #0 Loss at step 2885: 0.1764, time: 530.687. Train accuracy 0.864, Validation accuracy 0.714
{'accuracy': 0.7639793153936321, 'precision@5': 0.19583775282542704, 'recall@5': 0.08462963932068755, 'map@1': 0.23064262462967705, 'map@5': 0.13731892274118238, 'map@10': 0.11425578565504584, 'diversity@5': 0.1171658647277152, 'diversity@10': 0.12522683848654445, 'epc@5': 0.7473682398815811, 'epc@10': 0.7665372419631042, 'epd@5': 0.14907433103713885, 'coverage@1': 0.0633850303438975, 'coverage@5': 0.13226086118871014, 'coverage@10': 0.17454965802909161, 'name': 'NeuralLogicRec_AE_ML_6epochs', 'embedding_dim': 32, 'epochs_trained': 7, 'batch_size': 48, 'nr_hidden_layers': 3, 'nr_item_samples': 4096}
Epoch #0 Loss at step 2885: 0.1841, time: 531.785. Train accuracy 0.866, Validation accuracy 0.713
{'accuracy': 0.7704576561014852, 'precision@5': 0.19650341977250285, 'recall@5': 0.08311864163137603, 'map@1': 0.23539738853736147, 'map@5': 0.13850949327221226, 'map@10': 0.11447389975968524, 'd

In [83]:
nlr_ml.additional_name = 'AE_ML_10_epochs'
nlr_ml.save('../saved_models')

In [110]:
pd.DataFrame(nlr_history)

,accuracy,batch_size,coverage@1,coverage@10,coverage@5,diversity@10,diversity@5,embedding_dim,epc@10,epc@5,epd@5,epochs_trained,map@1,map@10,map@5,name,nr_hidden_layers,nr_item_samples,precision@5,recall@5
0,0.662211,48,0.018206,0.059724,0.039399,0.127913,0.120962,32,0.716507,0.688496,0.155785,1,0.156249,0.070051,0.084325,NeuralLogicRec_default,3,4096,0.128166,0.057129
1,0.741166,48,0.029284,0.090357,0.063192,0.123090,0.115783,32,0.734332,0.710480,0.152213,2,0.199627,0.098099,0.117074,NeuralLogicRec_default,3,4096,0.171559,0.074401
2,0.756827,48,0.043252,0.125710,0.091706,0.121741,0.114167,32,0.749312,0.729191,0.150313,3,0.223547,0.107418,0.129927,NeuralLogicRec_default,3,4096,0.184960,0.079643
3,0.752100,48,0.050958,0.151238,0.108660,0.123381,0.115557,32,0.759435,0.740621,0.149968,4,0.223291,0.107024,0.129669,NeuralLogicRec_default,3,4096,0.186094,0.078803
4,0.759215,48,0.055871,0.156632,0.115692,0.124262,0.116207,32,0.767957,0.753111,0.150008,5,0.228594,0.108817,0.132118,NeuralLogicRec_default,3,4096,0.187850,0.078178
5,0.782511,48,0.056642,0.162990,0.119256,0.123521,0.115019,32,0.754347,0.734855,0.148637,6,0.237702,0.114901,0.138578,NeuralLogicRec_default,3,4096,0.195318,0.083840
6,0.763979,48,0.063385,0.174550,0.132261,0.125227,0.117166,32,0.766537,0.747368,0.149074,7,0.230643,0.114256,0.137319,NeuralLogicRec_AE_ML_6epochs,3,4096,0.195838,0.084630
7,0.770458,48,0.062229,0.173297,0.127155,0.123551,0.115431,32,0.754862,0.733586,0.148433,8,0.235397,0.114474,0.138509,NeuralLogicRec_AE_ML_6epochs,3,4096,0.196503,0.083119
8,0.782637,48,0.060110,0.175513,0.128600,0.124598,0.116239,32,0.751061,0.731217,0.149382,9,0.243005,0.119432,0.143328,NeuralLogicRec_AE_ML_6epochs,3,4096,0.201587,0.086136
9,0.763969,48,0.062422,0.181485,0.136307,0.124630,0.116813,32,0.768870,0.751723,0.149664,10,0.233861,0.113799,0.137675,NeuralLogicRec_AE_ML_6epochs,3,4096,0.195626,0.082952


In [17]:
nlr_ml_nn = models.NeuralLogicRec.NLR(utils.common.movie_lens['user'], utils.common.movie_lens['dimensions'], epochs=1, constraints=constraints, mode='nn', name='NN_ML_10_epochs')
nlr_history_nn = list()
for e in range(10):
    nlr_ml_nn.train(utils.common.load_dataset(utils.common.movie_lens), utils.common.movie_lens['train']['records'])
    nlr_history_nn.append(eval_ml.evaluate_single_thread(nlr_ml_nn))
nlr_ml_nn.save('../saved_models')

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch #0 Loss at step 2885: 0.3053, time: 860.654. Train accuracy 0.802, Validation accuracy 0.656
Epoch #0 Loss at step 2885: 0.2906, time: 850.057. Train accuracy 0.817, Validation accuracy 0.693
Epoch #0 Loss at step 2885: 0.2675, time: 849.426. Train accuracy 0.808, Validation accuracy 0.619
Epoch #0 Loss at step 2885: 0.2895, time: 847.871. Train accuracy 0.817, Validation accuracy 0.678
Epoch #0 Loss at step 2885: 0.2747, time: 846.828. Train accuracy 0.818, Validation accuracy 0.706
Epoch #0 Loss at step 2885: 0.3012, time: 846.495. Train accuracy 0.820, Validation accuracy 0.685
Epoch #0 Loss at step 2885: 0.2642, time: 845.379. Train accuracy 0.822, Validation accuracy 0.680
Epoch #0 Loss at step 2885: 0.2683, time: 846.312. Train accuracy 0.803, Validation accuracy 0.646
Epoch #0 Loss at step 2885: 0.2453, time: 845.316. Train accuracy 0.831, Validation accuracy 0.734
Epoch #0 Loss at step 2885: 0.2565, time: 845.620. Train accuracy 0.801, Validation accuracy 0.663


In [7]:
nlr_ml_v2 = models.NeuralLogicRec.NLR(utils.common.movie_lens['user'], utils.common.movie_lens['dimensions'], epochs=1, constraints=constraints, mode='v2', name='V2_ML_10_epochs')
nlr_history_v2 = list()
for e in range(10):
    nlr_ml_v2.train(utils.common.load_dataset(utils.common.movie_lens), utils.common.movie_lens['train']['records'])
    ev_result = eval_ml.evaluate_single_thread(nlr_ml_v2)
    nlr_history_v2.append(ev_result)
    pd.DataFrame(nlr_history_v2).to_csv('../evals/V2_ML_10_epochs.csv')
nlr_ml_nn.save('../saved_models/NLR')

W0912 08:16:38.986819 140244721764160 deprecation.py:323] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow_core/python/ops/math_grad.py:1424: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch #1 Loss at step 5770: 0.1123, time: 347.705. Train P@1 0.466 P@5 0.431, Eval P@1 0.022 P@5 0.025
Epoch #1 Loss at step 5770: 0.1003, time: 334.177. Train P@1 0.540 P@5 0.481, Eval P@1 0.024 P@5 0.027
Epoch #1 Loss at step 5770: 0.0971, time: 335.844. Train P@1 0.542 P@5 0.485, Eval P@1 0.028 P@5 0.029
Epoch #1 Loss at step 5770: 0.0955, time: 331.848. Train P@1 0.516 P@5 0.491, Eval P@1 0.037 P@5 0.027
Epoch #1 Loss at step 5770: 0.0944, time: 333.180. Train P@1 0.573 P@5 0.506, Eval P@1 0.025 P@5 0.029
Epoch #1 Loss at step 5770: 0.0937, time: 334.129. Train P@1 0.554 P@5 0.501, Eval P@1 0.032 P@5 0.027
Epoch #1 Loss at step 5770: 0.0931, time: 333.900. Train P@1 0.524 P@5 0.505, Eval P@1 0.029 P@5 0.028
Epoch #1 Loss at step 5770: 0.0926, time: 335.217. Train P@1 0.565 P@5 0.518, Eval P@1 0.031 P@5 0.031
Epoch #1 Loss at step 5770: 0.0923, time: 328.943. Train P@1 0.528 P@5 0.501, Eval P@1 0.029 P@5 0.027
Epoch #1 Loss at step 5770: 0.0921, time: 329.253. Train P@1 0.520 P@5 0.

NameError: name 'nlr_ml_nn' is not defined

In [22]:
pd.DataFrame(nlr_history_v2).to_csv('../evals/V2_ML_10_epochs.csv')

### Comparison of model architectures on MovieLens data

## Models for MSD data

In [8]:
eval_msd = evaluation.Evaluation(utils.common.msd)
# nlr_msd_ae = models.NeuralLogicRec.NLR(utils.common.msd['user'], utils.common.msd['dimensions'], epochs=1, constraints=constraints, mode='ae', name='AE_MSD_10_epochs')


In [9]:
constraints_msd = list()
constraints_msd.append(Constraint(weight=0.2, formula=item_cf))
constraints_msd.append(Constraint(weight=0.2, formula=user_cf))
@tf.function
def likes_equiv(model, outputs):
    return Forall(Equiv(outputs['rec'], outputs['likes']))
constraints_msd.append(Constraint(weight=0.8, formula=likes_equiv))

popularity_msd = np.load(utils.common.msd['train']['item_frequency'])
popularity_msd = tf.convert_to_tensor(popularity_msd.squeeze(), tf.float32)
@tf.function
def novelty_constraint(model, outputs):
    return Forall(Implies(popularity_msd, Not(outputs['rec'])))
constraints_msd.append(Constraint(weight=0.3, formula=novelty_constraint))
constraints_msd.append(Constraint(weight=0.3, formula=diversity_constraint))

In [26]:
nlr_history_msd_ae = list()
for e in range(10):
    nlr_msd_ae.train(utils.common.load_dataset(utils.common.msd), utils.common.msd['train']['records'])
    nlr_history_msd_ae.append(eval_msd.evaluate_single_thread(nlr_msd_ae))
nlr_msd_ae.save('../saved_models')
pd.DataFrame(nlr_history_msd_ae).to_csv('../evals/AE_MSD_10_epochs.csv')

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch #0 Loss at step 2457: 0.3340, time: 328.467. Train accuracy 0.724, Validation accuracy 0.465
Epoch #0 Loss at step 2457: 0.2783, time: 320.835. Train accuracy 0.788, Validation accuracy 0.575
Epoch #0 Loss at step 2457: 0.2890, time: 322.371. Train accuracy 0.777, Validation accuracy 0.585
Epoch #0 Loss at step 2457: 0.2493, time: 321.966. Train accuracy 0.787, Validation accuracy 0.538
Epoch #0 Loss at step 2457: 0.2530, time: 321.648. Train accuracy 0.797, Validation accuracy 0.628
Epoch #0 Loss at step 2457: 0.2628, time: 321.805. Train accuracy 0.783, Validation accuracy 0.490
Epoch #0 Loss at step 2457: 0.2583, time: 321.144. Train accuracy 0.799, Validation accuracy 0.614
Epoch #0 Loss at step 2457: 0.2412, time: 322.453. Train accuracy 0.813, Validation accuracy 0.707
Epoch #0 Loss at step 2457: 0.2473, time: 321.494. Train accuracy 0.790, Validation accuracy 0.601
Epoch #0 Loss at step 2457: 0.2455, time: 322.070. Train accuracy 0.799, Validation accuracy 0.557


In [41]:
nlr_msd_nn = models.NeuralLogicRec.NLR(utils.common.msd['user'], utils.common.msd['dimensions'], epochs=1, constraints=constraints, mode='nn', name='NN_MSD_10_epochs')
nlr_history_msd_nn = list()
for e in range(10):
    nlr_msd_nn.train(utils.common.load_dataset(utils.common.msd), utils.common.msd['train']['records'])
    nlr_history_msd_nn.append(eval_msd.evaluate_single_thread(nlr_msd_nn))
nlr_msd_nn.save('../saved_models')
pd.DataFrame(nlr_history_msd_nn).to_csv('../evals/NN_MSD_10_epochs.csv')

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch #0 Loss at step 2457: 0.4523, time: 503.762. Train accuracy 0.700, Validation accuracy 0.533
Epoch #0 Loss at step 2457: 0.4498, time: 493.903. Train accuracy 0.684, Validation accuracy 0.423
Epoch #0 Loss at step 2457: 0.4354, time: 493.372. Train accuracy 0.679, Validation accuracy 0.462
Epoch #0 Loss at step 2457: 0.3933, time: 494.941. Train accuracy 0.703, Validation accuracy 0.416
Epoch #0 Loss at step 2457: 0.3980, time: 496.310. Train accuracy 0.686, Validation accuracy 0.410
Epoch #0 Loss at step 2457: 0.3974, time: 494.934. Train accuracy 0.697, Validation accuracy 0.468
Epoch #0 Loss at step 2457: 0.4141, time: 497.709. Train accuracy 0.696, Validation accuracy 0.504
Epoch #0 Loss at step 2457: 0.4180, time: 497.081. Train accuracy 0.717, Validation accuracy 0.545
Epoch #0 Loss at step 2457: 0.4069, time: 493.295. Train accuracy 0.750, Validation accuracy 0.579
Epoch #0 Loss at step 2457: 0.3956, time: 493.245. Train accuracy 0.713, Validation accuracy 0.531


In [11]:
nlr_msd_v2 = models.NeuralLogicRec.NLR(utils.common.msd['user'], utils.common.msd['dimensions'], epochs=1, constraints=constraints_msd, mode='v2', name='V2_MSD_10_epochs')
nlr_history_msd_v2 = list()
for e in range(10):
    nlr_msd_v2.train(utils.common.load_dataset(utils.common.msd), utils.common.msd['train']['records'])
    nlr_history_msd_v2.append(eval_msd.evaluate_single_thread(nlr_msd_v2))
    pd.DataFrame(nlr_history_msd_v2).to_csv('../evals/V2_MSD_10_epochs.csv')
nlr_msd_v2.save('../saved_models/NLR')
pd.DataFrame(nlr_history_msd_v2).to_csv('../evals/V2_MSD_10_epochs.csv')

Epoch #1 Loss at step 4915: 0.1596, time: 258.066. Train P@1 0.252 P@5 0.246, Eval P@1 0.014 P@5 0.013
Epoch #1 Loss at step 4915: 0.1342, time: 251.878. Train P@1 0.352 P@5 0.307, Eval P@1 0.018 P@5 0.017
Epoch #1 Loss at step 4915: 0.1282, time: 255.118. Train P@1 0.330 P@5 0.302, Eval P@1 0.022 P@5 0.019
Epoch #1 Loss at step 4915: 0.1249, time: 253.772. Train P@1 0.338 P@5 0.312, Eval P@1 0.017 P@5 0.014
Epoch #1 Loss at step 4915: 0.1228, time: 251.704. Train P@1 0.338 P@5 0.317, Eval P@1 0.019 P@5 0.016
Epoch #1 Loss at step 4915: 0.1213, time: 250.724. Train P@1 0.361 P@5 0.319, Eval P@1 0.026 P@5 0.019
Epoch #1 Loss at step 4915: 0.1203, time: 251.327. Train P@1 0.349 P@5 0.314, Eval P@1 0.018 P@5 0.015
Epoch #1 Loss at step 4915: 0.1195, time: 252.279. Train P@1 0.370 P@5 0.325, Eval P@1 0.023 P@5 0.019
Epoch #1 Loss at step 4915: 0.1188, time: 250.268. Train P@1 0.344 P@5 0.322, Eval P@1 0.021 P@5 0.018
Epoch #1 Loss at step 4915: 0.1183, time: 254.450. Train P@1 0.354 P@5 0.

# Constraint evaluation

In [3]:
base_constraints = list()
# base_constraints.append(Constraint(weight=0.25, formula=item_cf))
# base_constraints.append(Constraint(weight=0.75, formula=user_cf))
@tf.function
def likes_equiv(model, outputs):
    return Forall(Equiv(outputs['rec'], outputs['likes']))
base_constraints.append(Constraint(weight=0.95, formula=likes_equiv))
@tf.function
def novelty_constraint(model, outputs):
    return Forall(Implies(outputs['popular'], Not(outputs['rec'])))

In [26]:
def diversity_evaluation(dataset, ds_name):
    diversity_weights = [0.5, 1.0, 2.0, 4.0]
    for div_weight in diversity_weights:
        evals = list()
        div_constraint = Constraint(weight=div_weight, formula=diversity_constraint)
        diversity_model = models.NeuralLogicRec.NLR(dataset['user'], dataset['dimensions'], epochs=1, constraints=[div_constraint] + base_constraints, mode='v2', name=ds_name + '_div_v2_' + str(div_weight))
        for e in range(5):
            diversity_model.train(utils.common.load_dataset(dataset), dataset['train']['records'])
            if ds_name == 'MSD':
                evals.append(eval_msd.evaluate_single_thread(diversity_model))
            else:
                evals.append(eval_ml.evaluate_single_thread(diversity_model))
        diversity_model.model._distribution_stragey = None
        diversity_model.save('../saved_models/NLR')
        pd.DataFrame(evals).to_csv('../evals/' + diversity_model.additional_name + '.csv')

In [5]:
ml_pretrain = models.NeuralLogicRec.NLR(utils.common.movie_lens['user'], utils.common.movie_lens['dimensions'], epochs=5, constraints=base_constraints, mode='v2', name='pretrain_v2')
ml_pretrain.model._distribution_strategy = None
ml_pretrain.train(utils.common.load_dataset(utils.common.movie_lens), utils.common.movie_lens['train']['records'])
ml_pretrain.save('../saved_models/NLR')

W0906 15:39:30.339435 140675917379392 deprecation.py:323] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow_core/python/ops/math_grad.py:1424: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch #1 Loss at step 5770: 0.1144, time: 320.540. Train P@1 0.468 P@5 0.439, Eval P@1 0.033 P@5 0.025
Epoch #2 Loss at step 5770: 0.1012, time: 312.830. Train P@1 0.517 P@5 0.470, Eval P@1 0.029 P@5 0.029
Epoch #3 Loss at step 5770: 0.0981, time: 311.357. Train P@1 0.549 P@5 0.495, Eval P@1 0.026 P@5 0.025
Epoch #4 Loss at step 5770: 0.0965, time: 313.219. Train P@1 0.532 P@5 0.503, Eval P@1 0.032 P@5 0.027
Epoch #5 Loss at step 5770: 0.0956, time: 312.860. Train P@1 0.532 P@5 0.495, Eval P@1 0.030 P@5 0.028


In [ ]:
ml_pretrain.model._distribution_strategy = None
diversity_evaluation(utils.common.msd, 'MSD')

Epoch #1 Loss at step 4915: 0.1634, time: 239.246. Train P@1 0.272 P@5 0.251, Eval P@1 0.018 P@5 0.015
Epoch #1 Loss at step 4915: 0.1378, time: 236.473. Train P@1 0.350 P@5 0.310, Eval P@1 0.023 P@5 0.020
Epoch #1 Loss at step 4915: 0.1319, time: 238.980. Train P@1 0.333 P@5 0.311, Eval P@1 0.018 P@5 0.018
Epoch #1 Loss at step 4915: 0.1285, time: 237.611. Train P@1 0.363 P@5 0.326, Eval P@1 0.020 P@5 0.018
Epoch #1 Loss at step 4915: 0.1263, time: 232.530. Train P@1 0.367 P@5 0.321, Eval P@1 0.013 P@5 0.018
Epoch #1 Loss at step 4915: 0.1632, time: 251.003. Train P@1 0.268 P@5 0.245, Eval P@1 0.011 P@5 0.016
Epoch #1 Loss at step 4915: 0.1395, time: 247.596. Train P@1 0.322 P@5 0.290, Eval P@1 0.026 P@5 0.020
Epoch #1 Loss at step 4915: 0.1343, time: 248.698. Train P@1 0.362 P@5 0.314, Eval P@1 0.018 P@5 0.016
Epoch #1 Loss at step 4915: 0.1316, time: 249.624. Train P@1 0.363 P@5 0.314, Eval P@1 0.018 P@5 0.016
Epoch #1 Loss at step 4915: 0.1299, time: 243.345. Train P@1 0.362 P@5 0.

In [47]:
msd_pretrain = models.NeuralLogicRec.NLR(utils.common.msd['user'], utils.common.msd['dimensions'], epochs=5, constraints=base_constraints, mode='ae', name='msd_pretrain')
# msd_pretrain.train(utils.common.load_dataset(utils.common.msd), utils.common.msd['train']['records'])
msd_pretrain.model._distribution_stragey = None
# msd_pretrain.save('../saved_models/NLR')

In [49]:
msd_pretrain.model._distribution_strategy = None
constraint_evaluation(utils.common.msd, 'MSD', msd_pretrain, 'NeuralLogicRec_msd_pretrain')

Epoch #0 Loss at step 2457: 0.2512, time: 343.148. Train accuracy 0.817, Validation accuracy 0.724
Epoch #0 Loss at step 2457: 0.2371, time: 332.964. Train accuracy 0.819, Validation accuracy 0.659
Epoch #0 Loss at step 2457: 0.2577, time: 335.278. Train accuracy 0.838, Validation accuracy 0.700
Epoch #0 Loss at step 2457: 0.2184, time: 335.431. Train accuracy 0.814, Validation accuracy 0.686
Epoch #0 Loss at step 2457: 0.2408, time: 336.831. Train accuracy 0.814, Validation accuracy 0.646
Epoch #0 Loss at step 2457: 0.2331, time: 336.646. Train accuracy 0.837, Validation accuracy 0.720
Epoch #0 Loss at step 2457: 0.2579, time: 335.109. Train accuracy 0.815, Validation accuracy 0.650
Epoch #0 Loss at step 2457: 0.2229, time: 333.546. Train accuracy 0.803, Validation accuracy 0.685


In [4]:
def novelty_evalution(dataset, ds_name, nov_constraint_formula):
    novelty_weights = [0.1, 0.25, 0.5, 1.0]
    for nov_weight in novelty_weights:
        evals = list()
        nov_constraint = Constraint(weight=nov_weight, formula=nov_constraint_formula)
        nov_model = models.NeuralLogicRec.NLR(dataset['user'], dataset['dimensions'], epochs=1, constraints=[nov_constraint] + base_constraints, mode='v2', name=ds_name + '_nov_v2_' + str(nov_weight))
        for e in range(5):
            nov_model.train(utils.common.load_dataset(dataset), dataset['train']['records'])
            if ds_name == 'MSD':
                evals.append(eval_msd.evaluate_single_thread(nov_model))
            else:
                evals.append(eval_ml.evaluate_single_thread(nov_model))
        nov_model.model._distribution_stragey = None
        nov_model.save('../saved_models/NLR')
        pd.DataFrame(evals).to_csv('../evals/' + nov_model.additional_name + '.csv')

In [12]:
popularity = np.load(utils.common.movie_lens['train']['item_frequency'])
pop = tf.convert_to_tensor(popularity.squeeze(), tf.float32)
@tf.function
def ml_novelty_constraint(model, outputs):
    return Forall(Implies(pop, Not(outputs['rec'])))
novelty_evalution(utils.common.movie_lens, 'ML', ml_novelty_constraint)

Epoch #1 Loss at step 5770: 0.1076, time: 135.075. Train P@1 0.377 P@5 0.368, Eval P@1 0.023 P@5 0.024
Epoch #1 Loss at step 5770: 0.0965, time: 132.967. Train P@1 0.477 P@5 0.433, Eval P@1 0.026 P@5 0.026
Epoch #1 Loss at step 5770: 0.0932, time: 135.761. Train P@1 0.507 P@5 0.463, Eval P@1 0.028 P@5 0.029
Epoch #1 Loss at step 5770: 0.0913, time: 132.487. Train P@1 0.506 P@5 0.469, Eval P@1 0.039 P@5 0.026
Epoch #1 Loss at step 5770: 0.0901, time: 132.311. Train P@1 0.503 P@5 0.484, Eval P@1 0.037 P@5 0.026
Epoch #1 Loss at step 5770: 0.1081, time: 134.935. Train P@1 0.417 P@5 0.366, Eval P@1 0.027 P@5 0.025
Epoch #1 Loss at step 5770: 0.0966, time: 138.114. Train P@1 0.500 P@5 0.451, Eval P@1 0.028 P@5 0.027
Epoch #1 Loss at step 5770: 0.0934, time: 133.213. Train P@1 0.500 P@5 0.473, Eval P@1 0.034 P@5 0.025
Epoch #1 Loss at step 5770: 0.0917, time: 132.529. Train P@1 0.519 P@5 0.494, Eval P@1 0.036 P@5 0.026
Epoch #1 Loss at step 5770: 0.0903, time: 132.641. Train P@1 0.542 P@5 0.

In [7]:
popularity_msd = np.load(utils.common.msd['train']['item_frequency'])
popularity_msd = tf.convert_to_tensor(popularity_msd.squeeze(), tf.float32)
@tf.function
def msd_novelty_constraint(model, outputs):
    return Forall(Implies(popularity_msd, Not(outputs['rec'])))
novelty_evalution(utils.common.msd, 'MSD', msd_novelty_constraint)

Epoch #1 Loss at step 4915: 0.1539, time: 114.268. Train P@1 0.221 P@5 0.204, Eval P@1 0.012 P@5 0.012
Epoch #1 Loss at step 4915: 0.1272, time: 110.984. Train P@1 0.291 P@5 0.253, Eval P@1 0.013 P@5 0.015
Epoch #1 Loss at step 4915: 0.1202, time: 110.944. Train P@1 0.318 P@5 0.287, Eval P@1 0.032 P@5 0.020
Epoch #1 Loss at step 4915: 0.1160, time: 113.201. Train P@1 0.341 P@5 0.296, Eval P@1 0.019 P@5 0.016
Epoch #1 Loss at step 4915: 0.1134, time: 112.571. Train P@1 0.363 P@5 0.315, Eval P@1 0.018 P@5 0.016
Epoch #1 Loss at step 4915: 0.1518, time: 110.555. Train P@1 0.223 P@5 0.203, Eval P@1 0.013 P@5 0.013
Epoch #1 Loss at step 4915: 0.1265, time: 111.172. Train P@1 0.313 P@5 0.277, Eval P@1 0.014 P@5 0.016
Epoch #1 Loss at step 4915: 0.1202, time: 112.658. Train P@1 0.324 P@5 0.282, Eval P@1 0.013 P@5 0.015
Epoch #1 Loss at step 4915: 0.1172, time: 113.763. Train P@1 0.307 P@5 0.280, Eval P@1 0.019 P@5 0.014
Epoch #1 Loss at step 4915: 0.1153, time: 113.572. Train P@1 0.297 P@5 0.

Epoch #0 Loss at step 2457: 0.2415, time: 334.312. Train accuracy 0.817, Validation accuracy 0.732
Epoch #0 Loss at step 2457: 0.2650, time: 337.237. Train accuracy 0.814, Validation accuracy 0.608
Epoch #0 Loss at step 2457: 0.2491, time: 335.453. Train accuracy 0.828, Validation accuracy 0.711
Epoch #0 Loss at step 2457: 0.2502, time: 335.168. Train accuracy 0.795, Validation accuracy 0.677
Epoch #0 Loss at step 2457: 0.2477, time: 335.165. Train accuracy 0.808, Validation accuracy 0.698
Epoch #0 Loss at step 2457: 0.2524, time: 336.234. Train accuracy 0.795, Validation accuracy 0.626
Epoch #0 Loss at step 2457: 0.2340, time: 336.660. Train accuracy 0.817, Validation accuracy 0.604
Epoch #0 Loss at step 2457: 0.2466, time: 335.393. Train accuracy 0.838, Validation accuracy 0.689
Epoch #0 Loss at step 2457: 0.2288, time: 334.526. Train accuracy 0.837, Validation accuracy 0.726
Epoch #0 Loss at step 2457: 0.2361, time: 335.579. Train accuracy 0.829, Validation accuracy 0.636
Epoch #0 L

In [59]:
novelty_evalution(utils.common.movie_lens, 'ML', ml_pretrain, 'NeuralLogicRec_pretrain')

Epoch #0 Loss at step 2885: 0.1795, time: 557.274. Train accuracy 0.861, Validation accuracy 0.799
Epoch #0 Loss at step 2885: 0.2217, time: 548.487. Train accuracy 0.834, Validation accuracy 0.706
Epoch #0 Loss at step 2885: 0.1775, time: 551.901. Train accuracy 0.866, Validation accuracy 0.760
Epoch #0 Loss at step 2885: 0.2061, time: 546.200. Train accuracy 0.872, Validation accuracy 0.743
Epoch #0 Loss at step 2885: 0.1933, time: 546.090. Train accuracy 0.841, Validation accuracy 0.625
Epoch #0 Loss at step 2885: 0.1972, time: 545.309. Train accuracy 0.835, Validation accuracy 0.702
Epoch #0 Loss at step 2885: 0.1890, time: 546.188. Train accuracy 0.829, Validation accuracy 0.690
Epoch #0 Loss at step 2885: 0.2529, time: 546.056. Train accuracy 0.865, Validation accuracy 0.758
Epoch #0 Loss at step 2885: 0.2130, time: 547.248. Train accuracy 0.836, Validation accuracy 0.627
Epoch #0 Loss at step 2885: 0.1946, time: 546.237. Train accuracy 0.865, Validation accuracy 0.668
Epoch #0 L

In [10]:
np.arange(100).reshape(50,2)[np.random.rand(50) > 0.9]

array([[ 0,  1],
       [26, 27],
       [84, 85]])

In [1]:
!jupyter nbconvert --to html NeuralLogicRecEvaluation.ipynb

[NbConvertApp] Converting notebook NeuralLogicRecEvaluation.ipynb to html
[NbConvertApp] Writing 345568 bytes to NeuralLogicRecEvaluation.html
